In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import skimage
import tensorflow as tf
import imgaug_slices as im
import slices
import slices_config

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Data directory
DATA_DIR = os.path.join(ROOT_DIR, "database")
# Train directory
TRAIN_DIR = os.path.join(DATA_DIR, "train")
# Validate directory
VAL_DIR = os.path.join(DATA_DIR, "validate")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log


# Directory to save logs and trained model
config = slices_config.Config1()
MODEL_DIR = os.path.join(os.path.join(ROOT_DIR, "logs"),config.NAME)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
config.display()

In [ ]:
dataset_train = slices.SlicesDataset()
dataset_val = slices.SlicesDataset()

In [ ]:
channels = ["base","mf"]

In [ ]:
dataset_train.load_slices(TRAIN_DIR, 3, 40, channels = channels)
dataset_train.prepare()
dataset_val.load_slices(VAL_DIR, 3, 10, channels = channels)
dataset_val.prepare()

In [ ]:
mean_pixels = np.zeros(len(channels))
for image_id in dataset_train.image_ids:
    image = dataset_train.load_image(image_id)
    for n in range(len(channels)):
        mean_pixels[n]+= np.mean(image[:,:,n])
mean_pixels = mean_pixels/len(dataset_train.image_ids)
config.MEAN_PIXEL = mean_pixels


In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask",
                               "conv1"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [ ]:
# Train all branches to restore unloaded layers.
model.train(dataset_train, dataset_val,
           learning_rate = config.LEARNING_RATE,
            epochs = 10,
            layers = 'all',
            augmentation = im.aug())

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
        learning_rate=config.LEARNING_RATE, 
        epochs=20, 
        layers='heads',
        augmentation = im.aug())

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=40, 
            layers="all",
           augmentation = im.aug())

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_slices.h5")
model.keras_model.save_weights(MODEL_PATH)
print("Saving trained weights to", MODEL_PATH)